In [1]:
import numpy as np #相当于将numpy重命名为容易记忆的np，numpy是python的数值计算工具
import re# 用于正则表达式的匹配操作，在问别人能解析、复杂字符串分析、信息提取是一个很有用的工具
import os # os是python的系统编程的操作模块，可以处理文件和目录
import pandas as pd #Pandas是Python下一个开源数据分析的
from numpy.lib.format import open_memmap

In [2]:
max_frame = 20
number_joint = 25 #14
number_column = number_joint*3 # 14 joint * 3(x,y,z)    # 90-2(delete 85,86)

# choose_joints = np.array([4, 21, 9, 10, 11, 5, 6, 7, 17, 18, 19, 13, 14, 15 ]) - 1 # start from 0
# select_column = []
# for i in range(14): # 14 body join( except waist)
#     select_column.append(0 + 3*choose_joints[i]) # select x
#     select_column.append(1 + 3*choose_joints[i]) # select y
#     select_column.append(2 + 3*choose_joints[i]) # select z (non use)

In [3]:
def walk2(dirname):
    list_ = []
    for root, dirs, files in os.walk(dirname):
        for filename in files:
            list_.append(os.path.join(root, filename))
    return list_

def reshape_data(dir_):
    
    list_ = walk2(dir_)
    data_total = []
    
    for i in list_:
        with open(i,'r',encoding='utf-8') as f:
            data = f.readlines()

        ## extract data format
        data_temp = [] 
        index = 0
        for j in data:
            temp = re.findall(r"-*\d+\.?\d*",j)
            # print('1111111111111',temp)
            if temp != []:
                if index %25 == 0:
                    data_temp.append([float(k) for k in temp])
                else:
                    data_temp[-1].extend([float(k) for k in temp])
                index += 1

        ## choose frame
        for m in range(len(data_temp)):
            data = np.array(data_temp[m])
            if(data.size) == 75:#3*25
                data_total.append(data)

    data_total=pd.DataFrame(data_total)

#     import ipdb; ipdb.set_trace()
#     data_total = data_total.loc[ : , select_column ] # [row, column]
    
    print('{} has been saved'.format(dir_))
    return data_total

In [4]:
type_data = 'test' #'train' # 'test'
base_path = 'F:/Master Project/Dataset/Original/25 joints/'
path = base_path + type_data+'_set/' #
folder_list = ['拍球','投球','传球','站立']
data_list = []

for folder_name in folder_list:
    data_list.append(path + folder_name)    
    
data_x = []
data_y = []

In [5]:
max_row = 1000000

out_path = 'F:/Master Project/Dataset/Extract_data/25 joints'
tmp_x = open_memmap(
        '{}/tmp_x.npy'.format(out_path),
        dtype='float32',    
        mode='w+',
        shape=(max_row, max_frame, number_column))

tmp_y = open_memmap(
        '{}/tmp_y.npy'.format(out_path),
        dtype='float32',    
        mode='w+',
        shape=(max_row, ))

In [6]:
count_f = 0
for i in range(len(data_list)):
    print(i)
    data = reshape_data(data_list[i])
    data = np.array(data)

    slide_window = np.zeros((max_frame, number_column))    
    
    for j in range(len(data)):
#         a.extend(data.iloc[j,1:])
        slide_window = slide_window[1:max_frame]
        slide_window = np.append(slide_window, np.array([data[j]]), axis=0 )
        tmp_x[count_f] = slide_window
        tmp_y[count_f] = i
        count_f += 1
        
#         data_x.append(slide_window)
#         data_y.append(i)
    print("len:", count_f)
    
# data_x = np.array(data_x)
# data_y = np.array(data_y)        


0
F:/Master Project/Dataset/Original/25 joints/train_set/拍球 has been saved
len: 257260
1
F:/Master Project/Dataset/Original/25 joints/train_set/投球 has been saved
len: 515089
2
F:/Master Project/Dataset/Original/25 joints/train_set/传球 has been saved
len: 710494
3
F:/Master Project/Dataset/Original/25 joints/train_set/站立 has been saved
len: 912031


In [7]:
fp_x = open_memmap(
        '{}/{}_x.npy'.format(out_path, type_data),
        dtype='float32',    
        mode='w+',
        shape=(count_f, max_frame, number_column))

fp_y = open_memmap(
        '{}/{}_y.npy'.format(out_path, type_data),
        dtype='float32',    
        mode='w+',
        shape=(count_f, ))

fp_x[:] = tmp_x[:count_f]
fp_y[:] = tmp_y[:count_f]

In [8]:
import os
os.remove('{}/tmp_x.npy'.format(out_path, type_data))
os.remove('{}/tmp_y.npy'.format(out_path, type_data))

PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'F:/Master Project/Dataset/Extract_data/25 joints/tmp_x.npy'

In [ ]:
# out_path = 'Extract_Data'
# import h5py
# import time
# start_time = time.time()

# fp_x = open_memmap(
#         '{}/{}_x.npy'.format(out_path, type_data),
#         dtype='float32',    
#         mode='w+',
#         shape=(data_x.shape[0], max_frame, number_column))

# fp_y = open_memmap(
#         '{}/{}_y.npy'.format(out_path, type_data),
#         dtype='float32',    
#         mode='w+',
#         shape=(data_x.shape[0], ))

# fp_x[:] = data_x[:]
# fp_y[:] = data_y[:]


# # fp_x = h5py.File('{}/{}_y.hdf5'.format(out_path, type_data), 'w')
# # dset = fp_x.create_dataset("default", data=data_x)

# # np.save('{}/{}_x.npy'.format(out_path, type_data), data_x)

# print("--- %s seconds ---" % (time.time() - start_time))

# # print(type(fp_x))
# # print(fp_x[200])
# print("finish")

In [ ]:
# fp_x[371044]

In [ ]:
# fp_x.base.resize( count_f * max_frame * number_column * 4 )
# fp_x.flush()
# # fp_x.reshape((count_f, max_frame, number_column))
# # fp_x.flush()

In [ ]:
# fp_x23 = fp_x2.reshape((count_f, max_frame, number_column))
# print(fp_x23[0])

In [ ]:
# fp_x2 = np.memmap('{}/{}_x.npy'.format(out_path, type_data), dtype='float32', mode='r')
# # fp_x23 = fp_x2.reshape((count_f, max_frame, number_column))
# # fp_x23 = np.load('{}/{}_x.npy'.format(out_path, type_data), mmap_mode='r')
# print(fp_x2.shape)
# # print(fp_x23[1])
# print(fp_x2[829:850])

In [ ]:
# result = np.where(fp_x2 == 0.1808482)
# result